In [ ]:
# %pip install optuna

In [1]:
import shutil
import subprocess
import os
import time
import json

import pandas as pd
import numpy as np
from tqdm import tqdm

import spacy
# import optuna
from thinc.api import Config

In [2]:
COLAB = True

In [3]:
DATA_DIR = "gdrive/MyDrive/Work/quantify-news/data/" if COLAB else "data/"
PROJECT_DIR = "gdrive/MyDrive/Work/quantify-news/" if COLAB else "./"
TMP_DIR = "/content/" if COLAB else "./"

GPU_ID = "0" if COLAB else "-1"

DATA_TRAIN_BIN_PATH = DATA_DIR + "ner_train.spacy"
DATA_DEV_BIN_PATH = DATA_DIR + "ner_dev.spacy"
DATA_TEST_BIN_PATH = DATA_DIR + "ner_test.spacy"
PRED_BIN_PATH = DATA_DIR + "ner_pred.spacy"

SPACY_CONFIG_PATH = PROJECT_DIR + ("spacy_base_config_colab.cfg" if COLAB else "spacy_base_config.cfg")
SPACY_FULL_CONFIG_PATH = PROJECT_DIR + "spacy_config.cfg"

# For spacy training inner loop
TRAINED_MODEL_PATH = TMP_DIR + "models/"
BEST_MODEL_TRAIN_PATH = TRAINED_MODEL_PATH + "model-best/"
BEST_MODEL_RESULTS_PATH = BEST_MODEL_TRAIN_PATH + "meta.json"

# For optuna outer loop
BEST_MODEL_OPT_PATH = PROJECT_DIR + "models/model-best/"
METRICS_DIR = PROJECT_DIR + "metrics/"
METRICS_OUTPUT_PATH = METRICS_DIR + "metrics.json"
SAMPLE_OUTPUT_PATH = METRICS_DIR + "metrics_sample/"

HP_STUDY_NAME = "ner_hp"
HP_HISTORY_PATH = "sqlite:///{}/{}.db".format(PROJECT_DIR, HP_STUDY_NAME)
PARAMS_OUTPUT_PATH = PROJECT_DIR + "best_params.json"


In [4]:
if COLAB:
    from google.colab import drive
    drive.mount('/content/gdrive')
    assert spacy.require_gpu()

    import locale
    print(locale.getpreferredencoding())
    def getpreferredencoding(do_setlocale=True):
        return 'UTF-8'
    locale.getpreferredencoding = getpreferredencoding

Mounted at /content/gdrive
UTF-8


In [5]:
spacy_config = Config().from_disk(SPACY_CONFIG_PATH)

In [6]:
SPACY_BASE_MODEL = spacy_config['components']['ner']['source']

In [8]:
try:
    nlp_base = spacy.load(SPACY_BASE_MODEL)
except:
    spacy.cli.download(SPACY_BASE_MODEL)
    nlp_base = spacy.load(SPACY_BASE_MODEL)

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Fine-Tune Spacy

In [ ]:
# !python -m spacy init fill-config {SPACY_CONFIG_PATH} {SPACY_FULL_CONFIG_PATH}

✔ Auto-filled config with all values
✔ Saved config
gdrive/MyDrive/Work/quantify-news/spacy_config.cfg
You can now add your data and train your pipeline:
python -m spacy train spacy_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
# !python -m spacy train \
#     {SPACY_FULL_CONFIG_PATH} \
#     --gpu-id {GPU_ID} \
#     --output {TRAINED_MODEL_PATH} \
#     --verbose
# # --code ./entity_remapper.py \

✔ Created output directory: /content/models
ℹ Saving to output directory: /content/models
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2025-03-11 21:34:09,251] [INFO] Set up nlp object from config
[2025-03-11 21:34:09,265] [DEBUG] Loading corpus from path: gdrive/MyDrive/Work/quantify-news/data/ner_dev.spacy
[2025-03-11 21:34:09,267] [DEBUG] Loading corpus from path: gdrive/MyDrive/Work/quantify-news/data/ner_train.spacy
[2025-03-11 21:34:09,268] [INFO] Pipeline: ['ner']
[2025-03-11 21:34:09,268] [INFO] Resuming training for: ['ner']
[2025-03-11 21:34:09,276] [INFO] Created vocabulary
[2025-03-11 21:34:11,200] [INFO] Added vectors: en_core_web_md
[2025-03-11 21:34:11,307] [INFO] Finished initializing nlp object
[2025-03-11 21:34:11,307] [INFO] Initialized pipeline components: []
✔ Initialized pipeline

============================= Training pipeline =============================
[2025-03-11 21:34:11,322] [DEBUG] Loading corpus from path

## Hyperparameter tuning

In [ ]:
def shell(command, time_fmt=None):
    start = time.time()
    process = subprocess.Popen(command, shell=False,
                               stdout=subprocess.PIPE,
                               stderr=subprocess.STDOUT,
                               text=True,
                               encoding='utf-8',
                               bufsize=1)
    for line in process.stdout:
        print(line, end='', flush=True)
    process.wait()
    end = time.time()

    print(time_fmt.format(end - start))


In [ ]:
def objective(trial):
    start_size = trial.suggest_int("start_size", 100, 500)  # Tune batch start
    stop_size = trial.suggest_int("stop_size", 1000, 5000)  # Tune batch stop
    learning_rate = trial.suggest_float("lr", 1e-5, 1e-3, log=True)

    command = [
        "python", "-m", "spacy",
        "train", SPACY_FULL_CONFIG_PATH,
        "--gpu-id", GPU_ID,
        "--training.batcher.size.start", str(start_size),
        "--training.batcher.size.stop", str(stop_size),
        "--training.optimizer.learn_rate", str(learning_rate),
        "--output", TRAINED_MODEL_PATH,
    ]
    # TODO: I can use spacy.cli.train.train here instead of shell proc!
    shell(command, "Training time: {:.4f} sec")

    with open(BEST_MODEL_RESULTS_PATH) as fp:
        result = json.load(fp)

    return result['performance']['ents_f']



# class SaveBestModelCallback:
#     def __call__(self, study: optuna.study.Study, trial: optuna.trial.FrozenTrial) -> None:
#         if study.best_value <= trial.value:
#             shutil.copytree(BEST_MODEL_TRAIN_PATH, BEST_MODEL_OPT_PATH)

In [ ]:
# study = optuna.create_study(study_name=HP_STUDY_NAME, direction="maximize",
#                             storage=HP_HISTORY_PATH, load_if_exists=True,
#                             callbacks=[SaveBestModelCallback()])

In [ ]:
# study.optimize(objective, n_trials=20)

In [ ]:
# print("Best F1 (on val):")
# print(study.best_value)
# print("Best params:")
# print(study.best_params)

# Evaluate

## Visualize sample

In [ ]:
# if not os.path.exists(SAMPLE_OUTPUT_PATH):
#     os.makedirs(SAMPLE_OUTPUT_PATH)

In [ ]:
# !python -m spacy \
#   benchmark accuracy \
#   {BEST_MODEL_TRAIN_PATH} {DATA_DEV_BIN_PATH} \
#   --gpu-id {GPU_ID} \
#   --output {METRICS_OUTPUT_PATH} \
#   --displacy-path {SAMPLE_OUTPUT_PATH}

## Run and test confusion matrix

In [7]:
from spacy.tokens import DocBin

In [24]:
!python -m spacy apply \
    {BEST_MODEL_OPT_PATH} \
    {DATA_TRAIN_BIN_PATH} \
    {PRED_BIN_PATH} \
    --gpu-id {GPU_ID} \
    --force

ℹ Using GPU: 0
✔ Loaded model gdrive/MyDrive/Work/quantify-news/models/model-best/
3237it [00:58, 55.01it/s]


In [9]:
nlp = spacy.load(BEST_MODEL_OPT_PATH)

In [25]:
from spacy.tokens import DocBin
pred_docs = list(DocBin().from_disk(PRED_BIN_PATH).get_docs(nlp.vocab))

In [26]:
gold_docs = list(DocBin().from_disk(DATA_TRAIN_BIN_PATH).get_docs(nlp_base.vocab))

In [27]:
assert len(pred_docs) == len(gold_docs)

In [28]:
from spacy.scorer import Scorer
from spacy.training import Example

scorer = Scorer()
examples = [Example(infer, gold) for infer, gold in zip(pred_docs, gold_docs)]
scores = scorer.score(examples)
print("F1: ", scores['ents_f'])
print("Precision: ", scores['ents_p'])
print("Recall: ", scores['ents_r'])


F1:  0.9500461467466543
Precision:  0.9048456213602901
Recall:  1.0


In [29]:
with open(DATA_DIR + "debug_train.txt","wb") as f:
    for ex in examples:
        for ent in ex.predicted.ents:
            txt = ent.text + "\n"
            f.write(txt.encode(locale.getpreferredencoding()))

In [30]:
train_lines = set([])
with open(DATA_DIR + "debug_train.txt", "rb") as f:
    for line in f.readlines():
        train_lines.add(line.decode(locale.getpreferredencoding()))

In [31]:
dev_lines = set([])
with open(DATA_DIR + "debug_dev.txt", "rb") as f:
    for line in f.readlines():
        dev_lines.add(line.decode(locale.getpreferredencoding()))

In [32]:
len(train_lines - dev_lines)

6252

In [36]:
list(train_lines - dev_lines)[:100]

['9100 block of S. Perry,\n',
 '900 block of West Cullerton Street\n',
 '1400 block of North Leclaire\n',
 '7030 S. Halsted St.,\n',
 'Wilcox and Keeler Avenue,\n',
 '3900 block of West Madison\n',
 'Ridge Road and Calumet Avenue in Munster, Indiana,\n',
 '47th and Wolcott,\n',
 'Milwaukee and Kilbourn avenues.\n',
 '4700 block West Jackson.\n',
 '2300 block of North Merrimac,\n',
 '1000 block of West Barry\n',
 'Aberdeen Street from the south,\n',
 '200 block of South Karlov\n',
 '6100 block of South Whipple Street,\n',
 '100-block of North Leclaire in the Austin neighborhood\n',
 '4000 block of West Lawrence Avenue\n',
 '9100 block of South Woodlawn\n',
 'robbery\n',
 '200 block of North St. Louis Avenue\n',
 '3500 block of West Foster;\n',
 '3100 block of W. 57th Street;\n',
 '2000 block of South Michigan Avenue\n',
 'CICERO, Ill.\n',
 'Illinois Masonic Medical Center\n',
 '1300 block of West Hasting Street in the University Village\n',
 '4900 block of S Vincennes,\n',
 '43rd Street

In [33]:
len(dev_lines - train_lines)

814

In [37]:
list(dev_lines - train_lines)[:100]

['8900 block of South Parnell\n',
 '1600 block of South Paulina,\n',
 '1600 block of North Oakley Avenue\n',
 '4400 block of West 59th Street.\n',
 '4200 block of West 18th Street.\n',
 '4700 block of North Central Park Avenue,\n',
 '4200 block of North Hazel Street\n',
 '3100 block of South Springfield Avenue.\n',
 'Advocate Christ Medical Center in Oak Lawn,\n',
 '4600-block of South Drexel Blvd\n',
 'Avers and Elston.\n',
 '6500 block of South Drexel,\n',
 '15th Street and 49th Avenue,\n',
 '400 block of East McFetridge Drive\n',
 '6700 block of South State Street\n',
 'Melrose Avenue and Lafayette Street\n',
 '5200 block of South Wood;\n',
 '5100 block of South Calumet Avenue,\n',
 '6400 block of South Seeley,\n',
 '1438 W. 63rd St.,\n',
 '1600 block of West Barry Avenue in Lakeview\n',
 '100 block of East 75th  Street,\n',
 '2300 block of S. Blue Island\n',
 '68th and Throop\n',
 '2400 block of South Trumbull Avenue.\n',
 'first block of East Adams Street\n',
 'Kenwood.\n',
 '135t

In [34]:
len(train_lines & dev_lines)

275

In [38]:
list(train_lines & dev_lines)[:100]

['2400 block of South Wentworth\n',
 '300 block of South Cicero Avenue,\n',
 'North Lawndale neighborhood on the Southwest Side.\n',
 '8000 block of South Marshfield,\n',
 'South Side.\n',
 '2200 block of West Pershing,\n',
 '5100 block of North Troy\n',
 'CHICAGO\n',
 'Jarvis Avenue\n',
 '5200 block of South Michigan Avenue\n',
 'South Side Englewood\n',
 '5300 block of South Seeley,\n',
 '2800 block of West Catalpa.\n',
 'Back of the Yards.\n',
 '300 block of North Columbus\n',
 'Villa Park.\n',
 'Old Town\n',
 'Back of the Yards,\n',
 '200 block of North Avenue,\n',
 'Henderson Elementary School\n',
 'first block of West 111th Street,\n',
 'Fuller Park\n',
 '8800 block of South Eggleston,\n',
 'Near West Side\n',
 'Northwest Side.\n',
 'West Side,\n',
 '5000 block of West Chicago Avenue,\n',
 '**CHICAGO\n',
 '4800 block of South Bishop Street\n',
 '77th and Paulina,\n',
 'West Lawn neighborhood\n',
 '400 block of North Dearborn Street;\n',
 '225 W. Roosevelt Rd.,\n',
 'Gage Park\n',

XXX: this actually looks like it's doing well! it has great recall.
actually it doen't have any false negatives. and the false positives are all
IMO locations. so maybe don't need to do anything else?

Ok but i'm confused because i replicated this on the dev data and also didn't
get a false negative. but the trial didn't have 100% recall. so what is
going on here?

Scores on Train Set:
F1:  0.9500461467466543
Precision:  0.9048456213602901
Recall:  1.0